In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os, sys
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict, Counter
import pickle
import pdb
import datetime
from glob import glob
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Image as IImage


def show_dct(dct, n=3):
    return dict(list(dct.items())[:n])


pdata = Path('/data/utkface/')
p = Path('.')
from fastai import *
from fastai.vision import *

In [2]:
target = 'race'

In [3]:
df = pd.read_csv('img2targets.csv', usecols=['img_name', target, 'is_train'])
# df = pd.read_csv('img2targets.csv', usecols=['img_name', target, 'is_train']).sample(1000)

classes = sorted(set(df[target]))

def get_dataset(df, classes=None):
    return ImageClassificationDataset(df.img_name, df[target], classes)

train_ds = get_dataset(df[df.is_train==1], classes)
valid_ds = get_dataset(df[df.is_train==0], classes)

def get_data(sz, bs):
    return ImageDataBunch.create(train_ds, valid_ds, path=p, bs=bs, size=sz,
                            ds_tfms=get_transforms(), tfms=imagenet_norm)

In [ ]:
data = get_data(sz=200, bs=64)

i = np.random.choice(len(data.valid_ds))
img,label = data.valid_ds[i]

img.show(title=classes[label])

In [5]:
learn = ConvLearner(data, tvm.resnet50, metrics=accuracy)

In [7]:
def train_size(sz, bs, lr):
    learn.data=get_data(sz, bs)
    learn.freeze()
    learn.fit_one_cycle(5, slice(lr))
    learn.unfreeze()
    learn.fit_one_cycle(5, slice(lr/20, lr/2), pct_start=0.1)
    learn.save(f"{target}-{sz}")

In [ ]:
sz, bs = 200, 128 

learn.data=get_data(sz, bs)

learn.lr_find(start_lr=1e-6)
learn.recorder.plot()

In [ ]:
lr=

In [ ]:
train_size(sz, bs, lr)